In [1]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive
from VC_collections.AuthorityFiles import Authority_instance
from preprocess_1 import update_df_in_gdrive
import gspread
import pandas as pd

In [2]:
cred = Collection.get_google_drive_credentials()
client = gspread.authorize(cred)

# Create One big Dataframe of all tables

In [12]:
catalog_files = {
    "Architect": "1EhFxuphaOu-S8rZy_7O8OuLk3BMH_rAXjOTqDg5yMvY",
    "Dance": "1WcqUNkDhNbDyTdYLU_S3TbVkjZ6G03m36_9FqvBgaSc",
    "Design": "1M-vegxBcBI7J5hZvoqLkQQ7ATxMmfyyrFpAX4-vR7FM",
    "Theater": "1goBGosdUcCLzmdxIuF1SRYgk_mIT-eIsCJgdvd69rCw",
}


def create_df_from_gspread(client, branch):
    spread = client.open_by_key(catalog_files[branch])
    sheet = spread.worksheet("Master Catalog")
    df = pd.DataFrame(sheet.get_all_values())
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df.set_index("MMS id", inplace=True)
    return df

In [8]:
df_Architect = create_df_from_gspread(client, "Architect")

In [9]:
df_Dance = create_df_from_gspread(client, "Dance")

In [16]:
df_Design = create_df_from_gspread(client, "Design")

In [15]:
df_Theater = create_df_from_gspread(client, "Theater")

In [17]:
master_catalog = pd.concat([df_Architect, df_Dance, df_Design, df_Theater])

In [18]:
master_catalog.index = master_catalog.index.astype(str)

In [19]:
master_catalog.head()

,ארכיון,סימול,סימול אב,ברקוד,סימול מקורי,רמת תיאור,מספר מיכל,קוד תיק ארכיון,כותרת,כותרת אנגלית,...,חומרים קשורים,תאריך פקיעת החיסיון,תקציר,קרדיט דימוי1,קרדיט דימוי2,מספר מערכת דימוי מייצג,שם הקובץ בתיקיה של הפריט המייצג,קרדיט עברית,קרדיט אנגלית,סוג הארכיון
MMS id,,,,,,,,,,,,,,,,,,,,,
990049153800205171,ArBe,ArBe,,,,Fonds Record,,,אוסף דוד בסט,,...,,,,,,,,NaN,NaN,NaN
990049153810205171,ArBe,ArBe-001,ArBe,,,Sub-Fonds Record,,,תכניות ושרטוטים מגולגלים,Rolled-up floor plans and sketches,...,,,,,,,,NaN,NaN,NaN
990049153820205171,ArBe,ArBe-001-001,ArBe-001,202949,,Series Record,,,תכניות ושרטוטים - שכונת נוה דוד - דימונה 1 מתוך 2,Floor plans and sketches - Neve David neighbor...,...,,,,,,,,NaN,NaN,NaN
990049153830205171,ArBe,ArBe-001-002,ArBe-001,3788,,Series Record,,,תכניות ושרטוטים - שכונת נוה דוד - דימונה 2 מתוך 2,Floor plans and sketches - Neve David neighbor...,...,,,,,,,,NaN,NaN,NaN
990049153840205171,ArBe,ArBe-001-003,ArBe-001,202965,,Series Record,,,תכניות ושרטוטים - שיקום בתים - קונטרה 1 מתוך 2,Floor plans and sketches - Housing restoration...,...,,,,,,,,NaN,NaN,NaN


In [25]:
len(sorted(master_catalog["ארכיון"].unique()))

98

# get bad records

In [26]:
df_repairs = pd.ExcelFile("Data/copyright_repairs_30062021.xlsx").parse("Sheet1").set_index("001")

In [27]:
df_repairs.index = df_repairs.index.rename("MMS id")
    

In [28]:
df_repairs.index = df_repairs.index.astype(str)

In [29]:
master_catalog.loc[df_repairs.index[0], ["תאריך מנורמל מוקדם", "תאריך מנורמל מאוחר", "סימול"]]

תאריך מנורמל מוקדם                 1963
תאריך מנורמל מאוחר           1986-08-26
סימול                 PZvHe-003-001-013
Name: 997007616462305171, dtype: object

In [30]:
df_repairs["תאריך מנורמל מוקדם"] = ''
df_repairs["תאריך מנורמל מאוחר"] = ''

for index, row in df_repairs.iterrows():
    if index in list(master_catalog.index):
        df_repairs.loc[index, "תאריך מנורמל מוקדם"] = master_catalog.loc[index, "תאריך מנורמל מוקדם"]
        df_repairs.loc[index, "תאריך מנורמל מאוחר"] = master_catalog.loc[index, "תאריך מנורמל מאוחר"]
    

In [31]:
df_repairs.to_excel("copyright_repairs_results_07072021.xlsx")

# Find Keywords

In [ ]:
from VC_collections import files

In [ ]:
keywords_spreadsheet = client.open_by_url(
            "https://docs.google.com/spreadsheets/d/1OeykES6h-vzNsdqtvn_TznyxY3rg5aoYEFahj6tVXBE"
        )

In [ ]:
df_terms, cols = files.create_df_from_gs(keywords_spreadsheet, "נושאים")

In [ ]:
terms_to_search = list(filter(None,df_terms["נושא רמה 3"].tolist()))
# terms_to_search = map()

In [ ]:
df_rei = pd.ExcelFile(r'Data/REI.xlsx').parse('Sheet1')

In [ ]:
df_rei.head()

In [ ]:
columns_to_save = ["סימול",
                   "רמת תיאור",
                   "כותרת",
                   "תיאור",
                   "תאריך חופשי",
                   "תאריך מנורמל מוקדם",
                   "תאריך מנורמל מאוחר",
                   "יוצרים",
                   "מילות מפתח - אישים",
                   "מילות מפתח - מוסדות",
                   "מילות מפתח_נושאים",
                   "מילות מפתח_מקומות"]

In [ ]:
from collections import defaultdict
dfs = defaultdict()
for term in terms_to_search:

    df = master_catalog[master_catalog.apply(lambda row: row.astype(str).str.contains(term).any(), axis=1)]
    dfs[term]= df[columns_to_save]
    

In [ ]:
from collections import defaultdict
dfs = defaultdict()

for term in terms_to_search:
    df = df_rei[df_rei.apply(lambda row: row.astype(str).str.contains(term).any(), axis=1)]
    dfs[term] = df

In [ ]:
occurences = dict()

for key in dfs.keys():
    occurences[key] = len(dfs[key])
    print(f"{key} - מספר מופעים: {len(dfs[key])}")



In [ ]:
df_terms

In [ ]:
df_occurances = pd.DataFrame(occurences, index=["occurances"]).transpose()
df_terms = df_terms.set_index("נושא רמה 3")
# for index, row in df_occurances.iterrows():
#     df_


In [ ]:
df_join = df_occurances.join(df_terms)
df_join.loc[:,["נושא רמה 1", "נושא רמה 2", "occurances"]].to_excel(input("name:")+".xlsx")

In [ ]:

keywords_spreadsheet = pd.ExcelWriter(input(str("enter file name without extension"))+'.xlsx')
for term, df in dfs.items():
    print(term)
    print(len(df))
    df.to_excel(keywords_spreadsheet, sheet_name=term.decode('utf-8','ignore').encode("utf-8"))

In [ ]:
from pandas import ExcelWriter


def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for term, df in list_dfs.items():
            df.to_excel(writer,f's_{term}')
        writer.save()

In [ ]:
save_xls(dfs, "keywords.xlsx")

# update correct names
from Python\Projects\Authorities

In [ ]:
import json

with open("../Projects/Authorities/names_variant.json", encoding='utf-8') as fp:
    names_dict = json.load(fp)

In [ ]:
dfs = [df_Architect, df_Dance, df_Design, df_Theater]


In [ ]:
from VC_collections.authorities import convert_dict

names_mapping_dict = convert_dict(names_dict)
names_mapping_dict = {key: value[0] for key, value in names_mapping_dict.items()} 

In [ ]:
%%timeit

for key in names_mapping_dict.keys():
    for df in dfs:
        df.replace(key, names_mapping_dict[key])

In [ ]:
from  df2gspread import df2gspread as d2g
from VC_collections.Collection import get_google_drive_credentials

In [ ]:
def update_master_catalog_gfile(df, gfile, name, credentials):
    wkt = d2g.upload(df, 
                     gfile=gfile,
                     wks_name=name,
                    credentials=credentials, 
                    )
    print( f'uploaded {eval(df)} to google sheets')

In [ ]:
cred = get_google_drive_credentials()


In [ ]:
update_master_catalog_gfile(df_Architect, catalog_files["Architect"], "test", credentials=cred)
update_master_catalog_gfile(df_Dance, catalog_files["Dance"], "test", credentials=cred)
update_master_catalog_gfile(df_Design, catalog_files["Design"], "test", credentials=cred)
update_master_catalog_gfile(df_Theater, catalog_files["Theater"], "test", credentials=cred)

## Look up creators in entire catalog

In [ ]:
import numpy as np

term_to_search = input(str("What to search? "))

mask = np.column_stack([master_catalog[col].astype(str).str.contains(term_to_search, na=False) for col in master_catalog])
df_test = master_catalog.loc[mask.any(axis=1)]

In [ ]:
df_test[['יוצרים', "ארכיון"]]

In [ ]:
import xmltodict 
import json

In [ ]:
with open(r"C:\Users\Yaelg\Google Drive\National_Library\Python\VC-Architect\PYeIl\Data\processed\PYeIl_final_20210617.xml", encoding="utf8") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [ ]:
xml_file.close()


In [ ]:
json_data = json.dumps(data_dict)

In [ ]:
with open("data.json", "w") as json_file:
        json_file.write(json_data)

In [ ]:
json_file.close()
